In [80]:
!pip install backtesting
!pip install pandas_ta
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas_ta as ta
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime

In [120]:
# ticker and the start and end dates for testing
ticker =  input("Enter ticker: ")
start = datetime.datetime(2013, 1, 1)
end = datetime.datetime(2023, 1, 1)

Enter ticker: xlp


In [121]:
df=yf.download( ticker , start=start,end=end)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-02,35.400002,35.810001,35.290001,35.799999,27.522751,10813900
2013-01-03,35.750000,35.820000,35.610001,35.709999,27.453562,6537300
2013-01-04,35.669998,35.869999,35.669998,35.810001,27.530443,5942500
2013-01-07,35.759998,35.790001,35.470001,35.570000,27.345934,3029900
2013-01-08,35.549999,35.549999,35.410000,35.470001,27.269054,4506100
...,...,...,...,...,...,...
2022-12-23,74.830002,75.230003,74.580002,75.180000,75.180000,4874700
2022-12-27,75.279999,75.690002,75.120003,75.540001,75.540001,4839700
2022-12-28,75.760002,75.860001,74.599998,74.599998,74.599998,5028900


In [122]:
def indicator(data):
  return data.Close.s.pct_change(periods = 7) * 100

In [123]:
class MomentumStrategy(Strategy):
  def init(self):
    self.pct_change = self.I(indicator, self.data)

  def next(self):
    change = self.pct_change[-1]
    
    if self.position:
      if change < 0:
        self.position.close()
    else:
      if change > 5 and self.pct_change[-2] > 5:
        self.buy()


In [124]:
bt=Backtest(df,MomentumStrategy,cash=100_000) 
stats=bt.run()
print(stats)

Start                     2013-01-02 00:00:00
End                       2022-12-30 00:00:00
Duration                   3649 days 00:00:00
Exposure Time [%]                    1.588562
Equity Final [$]                106857.013062
Equity Peak [$]                 111275.255966
Return [%]                           6.857013
Buy & Hold Return [%]              108.240236
Return (Ann.) [%]                    0.665949
Volatility (Ann.) [%]                2.960097
Sharpe Ratio                         0.224975
Sortino Ratio                        0.427409
Calmar Ratio                         0.130414
Max. Drawdown [%]                   -5.106428
Avg. Drawdown [%]                   -2.288261
Max. Drawdown Duration     1522 days 00:00:00
Avg. Drawdown Duration      504 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                      11.247867
Worst Trade [%]                     -2.867881
Avg. Trade [%]                    

In [98]:
bt.plot()

Row(id='7033', ...)

In [99]:
stats['_trades']  # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1063,329,336,94.029999,93.370003,-701.575783,-0.007019,2014-04-24,2014-05-05,11 days
1,1178,458,462,84.239998,85.860001,1908.363235,0.019231,2014-10-27,2014-10-31,4 days
2,1259,498,504,80.330002,78.730003,-2014.398079,-0.019918,2014-12-23,2015-01-02,10 days
3,1279,521,522,77.500000,75.250000,-2877.750000,-0.029032,2015-01-28,2015-01-29,1 days
4,1203,530,541,80.040001,80.389999,421.048164,0.004373,2015-02-10,2015-02-26,16 days
5,1513,674,677,63.930000,63.840000,-136.170231,-0.001408,2015-09-04,2015-09-10,6 days
6,1418,696,705,68.099998,67.720001,-538.836105,-0.005580,2015-10-07,2015-10-20,13 days
7,1381,718,722,69.540001,66.839996,-3728.706322,-0.038827,2015-11-06,2015-11-12,6 days
8,1613,775,777,57.240002,56.130001,-1790.430984,-0.019392,2016-02-01,2016-02-03,2 days
9,1485,799,812,60.930000,60.689999,-356.402493,-0.003939,2016-03-07,2016-03-24,17 days
